In [ ]:

# async def summarize_content_tool(content: List[Document]) -> str:
#     """
#     Description:
#     This function summarizes a list of documents using a state-based summarization pipeline. It extracts summaries from individual documents, merges them iteratively, and generates a final summary.
#
#     Parameters:
#     content (List[Document]): A list of Document objects to be summarized.
#
#     Returns:
#     A str containing the final summarized content.
#     """
#     async def generate_summary(state: SummaryState):
#         try:
#             response = await initialize_doc_parser_chain().ainvoke(state["content"])
#             return {"summaries": [response]}
#         except Exception as e:
#             logger.exception("Failed to generate summary")
#             raise e
#
#     async def collapse_summaries(state: OverallState):
#         try:
#             doc_lists = split_list_of_docs(
#                 state["collapsed_summaries"], length_function, config_settings.MAX_TOKENS
#             )
#             results = []
#             for doc_list in doc_lists:
#                 results.append(await acollapse_docs(doc_list, reduce_summary_chain().ainvoke))
#             return {"collapsed_summaries": results}
#         except Exception as e:
#             logger.exception("Failed to collapse summaries")
#             raise e
#
#     async def generate_final_summary(state: OverallState):
#         try:
#             response = await reduce_summary_chain().ainvoke(state["collapsed_summaries"])
#             return {"final_summary": response}
#         except Exception as e:
#             logger.exception("Failed to generate final summary")
#             raise e
#
#     try:
#         graph = StateGraph(OverallState)
#         graph.add_node("generate_summary", generate_summary)
#         graph.add_node("collect_summaries", collect_summaries)
#         graph.add_node("collapse_summaries", collapse_summaries)
#         graph.add_node("generate_final_summary", generate_final_summary)
#
#         graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
#         graph.add_edge("generate_summary", "collect_summaries")
#         graph.add_conditional_edges("collect_summaries", should_collapse)
#         graph.add_conditional_edges("collapse_summaries", should_collapse)
#         graph.add_edge("generate_final_summary", END)
#
#         app = graph.compile()
#
#         async for step in app.astream(
#                 {"contents": [doc.page_content for doc in content]},
#                 {"recursion_limit": 10},
#         ):
#             if "final_summary" in step:
#                 return step["final_summary"]
#
#         raise Exception("Failed to generate final summary")
#
#     except Exception as e:
#         logger.exception("Summarization tool failed")
#         raise e


In [ ]:

# async def summarize_content_tool(content: list[Document]) -> str:
#     async def generate_summary(state: SummaryState):
#         response = await initialize_doc_parser_chain.ainvoke(state["content"])
#         return {"summaries": [response]}
#
#     async def collapse_summaries(state: OverallState):
#         doc_lists = split_list_of_docs(
#             state["collapsed_summaries"], length_function, config_settings.MAX_TOKENS
#         )
#         results = []
#         for doc_list in doc_lists:
#             results.append(await acollapse_docs(doc_list, reduce_summary_chain.ainvoke))
#
#         return {"collapsed_summaries": results}
#
#     async def generate_final_summary(state: OverallState):
#         response = await reduce_summary_chain.ainvoke(state["collapsed_summaries"])
#         return {"final_summary": response}
#
#     graph = StateGraph(OverallState)
#     graph.add_node("generate_summary", generate_summary)  # same as before
#     graph.add_node("collect_summaries", collect_summaries)
#     graph.add_node("collapse_summaries", collapse_summaries)
#     graph.add_node("generate_final_summary", generate_final_summary)
#
#     # Edges:
#     graph.add_conditional_edges(START, map_summaries, ["generate_summary"])
#     graph.add_edge("generate_summary", "collect_summaries")
#     graph.add_conditional_edges("collect_summaries", should_collapse)
#     graph.add_conditional_edges("collapse_summaries", should_collapse)
#     graph.add_edge("generate_final_summary", END)
#
#     app = graph.compile()
#
#     async for step in app.astream(
#             {"contents": [doc.page_content for doc in content]},
#             {"recursion_limit": 10},
#     ):
#         print(list(step.keys()))
#

In [ ]:
import pprint

from langgraph.prebuilt import create_react_agent
import asyncio
from typing import List
from langchain_core.documents import Document
from loguru import logger

from langgraph.checkpoint.memory import MemorySaver
from domains.utils import get_chat_model

from langchain_core.messages import HumanMessage

from langgraph.graph import END, START, StateGraph
from domains.agents.models import QueryRequest, OverallState, SummaryState
from domains.agents.tools import qna_tool, information_extraction_tool, summarize_content_tool
from domains.settings import config_settings


async def orchestrator_agent(query: str) -> str:
    """
    Orchestrator agent that uses qna_tool, information_extraction_tool, and summarize_content_tool.

    Args:
        query: The query string.

    Returns:
        A string containing the final summary.
    """
    async def run_qna_tool(state: OverallState):
        try:
            request = QueryRequest(query=state['query'])
            documents = await qna_tool(request)
            return documents
        except Exception as e:
            logger.exception("Failed to run qna_tool")
            raise e

    async def run_information_extraction_tool(state: OverallState):
        try:
            documents = await information_extraction_tool(query=state['query'])
            return documents
        except Exception as e:
            logger.exception("Failed to run information_extraction_tool")
            raise e

    async def run_summarize_content_tool(state: OverallState):
        try:
            summary = await summarize_content_tool(state["documents"])
            return {"final_summary": summary}
        except Exception as e:
            logger.exception("Failed to run summarize_content_tool")
            raise e

    try:
        graph = StateGraph(OverallState)
        graph.add_node("run_qna_tool", run_qna_tool)
        graph.add_node("run_information_extraction_tool", run_information_extraction_tool)
        graph.add_node("run_summarize_content_tool", run_summarize_content_tool)

        graph.add_edge(START, "run_qna_tool")
        graph.add_conditional_edges("run_qna_tool", lambda state: "run_information_extraction_tool" if not state["documents"] else "run_summarize_content_tool")
        graph.add_edge("run_information_extraction_tool", "run_summarize_content_tool")
        graph.add_edge("run_summarize_content_tool", END)

        app = graph.compile()

        async for step in app.astream({}, {"recursion_limit": 10}):
            if "final_summary" in step:
                return step["final_summary"]

        raise Exception("Failed to generate final summary")
    except Exception as e:
        logger.exception("Orchestrator agent failed")
        raise e


memory = MemorySaver()

async def new(query: str, id):
    agent_executor=create_react_agent(
        model=get_chat_model(model_key="OPENAI_CHAT"),
        tools=[orchestrator_agent],
        checkpointer=memory,
    )

    config = {"configurable": {"thread_id": id}}
    async for step in agent_executor.astream(
            {"messages": [HumanMessage(content=query)]},
            config,
            stream_mode="values",
    ):
        step["messages"][-1].pretty_print()

